<a href="https://colab.research.google.com/github/iqbalyk/Marketing-Analyst/blob/main/marketing_analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df_customers = "https://raw.githubusercontent.com/iqbalyk/Marketing-Analyst/main/customers.csv"
df_products = "https://raw.githubusercontent.com/iqbalyk/Marketing-Analyst/main/products.csv"
df_transactions = "https://raw.githubusercontent.com/iqbalyk/Marketing-Analyst/main/transactions.csv"
df_campaigns = "https://raw.githubusercontent.com/iqbalyk/Marketing-Analyst/main/campaigns.csv"
df_spend = "https://raw.githubusercontent.com/iqbalyk/Marketing-Analyst/main/marketing_spend.csv"
df_targets = "https://raw.githubusercontent.com/iqbalyk/Marketing-Analyst/main/targets.csv"

customers = pd.read_csv(df_customers)
products = pd.read_csv(df_products)
transactions = pd.read_csv(df_transactions, parse_dates=["transaction_date"])
campaigns = pd.read_csv(df_campaigns, parse_dates=["start_date","end_date"])
marketing_spend = pd.read_csv(df_spend, parse_dates=["month"])
targets = pd.read_csv(df_targets, parse_dates=["month"])

#Cleaning Data 1 Missing Value karena campaign_id tidak selalu sama seperti transaction jadi mencari sampai 0 dengan mengisi data
transactions['campaign_id'] = transactions['campaign_id'].fillna("No Campaign")

#Cleaning Data 2
customers['join_date'] = pd.to_datetime(customers['join_date'])
transactions['amount'] = transactions['amount'].astype(float)

In [ ]:
tables = {
    "customers": customers,
    "products": products,
    "transactions": transactions,
    "campaigns": campaigns,
    "marketing_spend": marketing_spend,
    "targets": targets
}

for name, df in tables.items():
    print(f"\nData Types in {name} ")
    print(df.dtypes)



Data Types in customers 
customer_id            object
full_name              object
gender                 object
age                     int64
segment                object
city                   object
province               object
join_date      datetime64[ns]
dtype: object

Data Types in products 
product_id       object
product_name     object
category         object
brand            object
unit_price      float64
cost            float64
dtype: object

Data Types in transactions 
transaction_id              object
transaction_date    datetime64[ns]
customer_id                 object
product_id                  object
quantity                     int64
unit_price                 float64
amount                     float64
channel                     object
campaign_id                 object
city                        object
province                    object
dtype: object

Data Types in campaigns 
campaign_id              object
campaign_name            object
channel            

In [ ]:
def unique_values(df, cols):
    for col in cols:
        print(f"\n----- Unique values in {col} -----")
        print(df[col].unique())

# kolom channel untuk CallCenter ditambahkan spasi
transactions['channel'] = transactions['channel'].replace({
    "CallCenter": "Call Center"})
transactions['channel'] = transactions['channel'].str.title()

unique_values(customers, ["gender", "segment", "city", "province"])
unique_values(transactions, ["channel", "city", "province"])
unique_values(products, ["category", "brand"])
unique_values(campaigns, ["channel"])




----- Unique values in gender -----
['Male' 'Female']

----- Unique values in segment -----
['Silver' 'Bronze' 'Gold' 'Priority']

----- Unique values in city -----
['Banjarmasin' 'Jakarta' 'Madiun' 'Serang' 'Yogyakarta' 'Malang' 'Bandung'
 'Denpasar' 'Bogor' 'Bekasi' 'Medan' 'Pekanbaru' 'Surakarta' 'Semarang'
 'Tegal' 'Makassar' 'Cimahi' 'Tangerang' 'Manado' 'Palembang' 'Pontianak'
 'Surabaya']

----- Unique values in province -----
['Kalimantan' 'Jakarta' 'East Java' 'Banten' 'Yogyakarta' 'West Java'
 'Bali' 'Sumatra' 'Central Java' 'Sulawesi']

----- Unique values in channel -----
['Internet' 'Mobile' 'Call Center' 'Agent' 'Branch']

----- Unique values in city -----
['Malang' 'Cimahi' 'Denpasar' 'Jakarta' 'Bekasi' 'Bogor' 'Serang'
 'Tangerang' 'Medan' 'Surakarta' 'Tegal' 'Makassar' 'Semarang' 'Palembang'
 'Yogyakarta' 'Manado' 'Surabaya' 'Bandung' 'Pekanbaru' 'Pontianak'
 'Banjarmasin' 'Madiun']

----- Unique values in province -----
['East Java' 'West Java' 'Bali' 'Jakarta' 'Bant

In [ ]:
# VALIDASI TRANSACTIONS
print("\n===== VALIDATE transactions.quantity =====")
print(transactions['quantity'].describe())

print("\n===== VALIDATE transactions.unit_price =====")
print(transactions['unit_price'].describe())

print("\n===== VALIDATE transactions.amount =====")
print(transactions['amount'].describe())

# Cek apakah amount konsisten dengan quantity * unit_price
transactions['calc_amount'] = transactions['quantity'] * transactions['unit_price']
transactions['amount_diff'] = transactions['amount'] - transactions['calc_amount']

print("\n===== Difference between amount and quantity*unit_price =====")
print(transactions['amount_diff'].describe())


===== VALIDATE transactions.quantity =====
count    50000.000000
mean         2.202200
std          1.097934
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          9.000000
Name: quantity, dtype: float64

===== VALIDATE transactions.unit_price =====
count    5.000000e+04
mean     2.482160e+06
std      1.587290e+06
min      6.610000e+04
25%      1.081175e+06
50%      2.371100e+06
75%      3.688025e+06
max      6.483700e+06
Name: unit_price, dtype: float64

===== VALIDATE transactions.amount =====
count    5.000000e+04
mean     5.576389e+06
std      5.490159e+06
min      6.654500e+04
25%      1.936646e+06
50%      4.116208e+06
75%      7.808390e+06
max      1.787872e+08
Name: amount, dtype: float64

===== Difference between amount and quantity*unit_price =====
count    5.000000e+04
mean     1.097235e+05
std      2.564741e+06
min     -3.430000e+02
25%     -4.300000e+01
50%      0.000000e+00
75%      4.300000e+01
max      1.575876e+08
Name: am

In [ ]:
# Buat kolom selisih
transactions['amount_calc'] = transactions['quantity'] * transactions['unit_price']
transactions['amount_diff'] = transactions['amount'] - transactions['amount_calc']

# Menentukan threshold anomali
threshold = 1_000_000   # 1 juta selisih

# Transaksi yang berbeda lebih dari threshold
outliers = transactions[transactions['amount_diff'].abs() > threshold]

outliers.head()

,transaction_id,transaction_date,customer_id,product_id,quantity,unit_price,amount,channel,campaign_id,city,province,calc_amount,amount_diff,amount_calc
33,T100033,2024-11-02,C100258,P1069,3,2078700.0,34829651.0,Mobile,CAM223,Yogyakarta,Yogyakarta,6236100.0,28593551.0,6236100.0
118,T100118,2023-09-30,C102921,P1128,1,3463400.0,21305403.0,Branch,CAM230,Pekanbaru,Sumatra,3463400.0,17842003.0,3463400.0
155,T100155,2023-10-28,C100655,P1110,2,4466600.0,34397215.0,Mobile,No Campaign,Surakarta,Central Java,8933200.0,25464015.0,8933200.0
244,T100244,2023-04-01,C101338,P1072,1,913600.0,5161132.0,Mobile,No Campaign,Manado,Sulawesi,913600.0,4247532.0,913600.0
569,T100569,2024-08-01,C101056,P1117,1,3637200.0,33239046.0,Branch,CAM229,Bogor,West Java,3637200.0,29601846.0,3637200.0


In [ ]:
# Clustering dahulu
bins = [-999999999, -1000000, -1000, 1000, 1000000, 999999999]
labels = ["Very Under", "Under", "Exact", "Over", "Very Over"]

transactions["amount_category"] = pd.cut(transactions["amount_diff"], bins=bins, labels=labels)

transactions["amount_category"].value_counts()

,count
amount_category,
Exact,49822
Very Over,178
Very Under,0
Under,0
Over,0


In [ ]:
transactions_full = transactions.copy()
transactions_clean = transactions.copy()

# jadikan amountnya konsisten tanpa tambahan
transactions_clean['amount'] = transactions_clean['quantity'] * transactions_clean['unit_price']

# hapus kolom helper
transactions_clean = transactions_clean.drop(columns=["amount_calc", "amount_diff"], errors='ignore')

# simpen data ke csv
transactions_full.to_csv("transactions_full.csv", index=False)
transactions_clean.to_csv("transactions_clean.csv", index=False)